In [4]:
import pandas as pd

crashes = pd.read_csv('../data/crashes_crashes.csv')


In [5]:
crashes_prime_cause = crashes[(crashes['PRIM_CONTRIBUTORY_CAUSE'] != 'UNABLE TO DETERMINE') & (crashes['PRIM_CONTRIBUTORY_CAUSE'] != 'NOT APPLICABLE')]

crashes_prime_cause_filled = crashes_prime_cause.fillna({'INTERSECTION_RELATED_I': 'N', 'NOT_RIGHT_OF_WAY_I': 
                                                         'N', 'HIT_AND_RUN_I':'N'})

crashes_prime_cause_filleddrop = crashes_prime_cause_filled.dropna(subset=['LATITUDE', 'LONGITUDE', 'INJURIES_TOTAL', 'INJURIES_FATAL', 
                                                                           'MOST_SEVERE_INJURY'])


from sklearn.cluster import KMeans

n_clusters = 30  # Number of clusters to create
X = crashes_prime_cause_filleddrop[['LONGITUDE', 'LATITUDE']]

# Create a K-Means clustering model
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(X)

# Add cluster labels to your data
cluster_labels = kmeans.labels_
crashes_prime_cause_filleddrop['GEO_KMEANS_Cluster'] = cluster_labels





crashes_prime_cause_filleddrop = crashes_prime_cause_filleddrop.drop(columns = ['CRASH_DATE_EST_I', 'DEVICE_CONDITION', 'REPORT_TYPE', 'DATE_POLICE_NOTIFIED',
                                                          'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME','BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I', 
                                                          'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 
                                                          'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_NO_INDICATION', 
                                                          'INJURIES_UNKNOWN', 'LOCATION', 'LANE_CNT', 'CRASH_DATE', 
                                                          'INJURIES_REPORTED_NOT_EVIDENT', 'TRAFFIC_CONTROL_DEVICE', 'INJURIES_TOTAL', 
                                                          'INJURIES_FATAL'])
crashes_prime_cause_filleddrop.info()


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 442382 entries, 2 to 798905
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CRASH_RECORD_ID          442382 non-null  object 
 1   POSTED_SPEED_LIMIT       442382 non-null  int64  
 2   WEATHER_CONDITION        442382 non-null  object 
 3   LIGHTING_CONDITION       442382 non-null  object 
 4   FIRST_CRASH_TYPE         442382 non-null  object 
 5   TRAFFICWAY_TYPE          442382 non-null  object 
 6   ALIGNMENT                442382 non-null  object 
 7   ROADWAY_SURFACE_COND     442382 non-null  object 
 8   ROAD_DEFECT              442382 non-null  object 
 9   CRASH_TYPE               442382 non-null  object 
 10  INTERSECTION_RELATED_I   442382 non-null  object 
 11  NOT_RIGHT_OF_WAY_I       442382 non-null  object 
 12  HIT_AND_RUN_I            442382 non-null  object 
 13  DAMAGE                   442382 non-null  object 
 14  PRIM

/var/folders/bt/mf607s9n0xs6sr08g8c44w9c0000gn/T/ipykernel_19463/3319199705.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crashes_prime_cause_filleddrop['GEO_KMEANS_Cluster'] = cluster_labels


In [3]:
#Transforming to numerical data

crashes_prime_cause_filleddrop_dummy = pd.get_dummies(crashes_prime_cause_filleddrop, columns = ['WEATHER_CONDITION', 'LIGHTING_CONDITION',
                                                                                                 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 
                                                                                                 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 
                                                                                                 'SEC_CONTRIBUTORY_CAUSE', 'ROAD_DEFECT',
                                                                                                 'MOST_SEVERE_INJURY'], drop_first=True, 
                                                                                                  dtype=int)
                                                                                                 




#binary/tertiary
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in ['CRASH_TYPE','INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE']:
    crashes_prime_cause_filleddrop_dummy[col] = le.fit_transform(crashes_prime_cause_filleddrop_dummy[col])


crashes_cleaned = crashes_prime_cause_filleddrop_dummy


crashes_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 442382 entries, 2 to 798905
Columns: 127 entries, CRASH_RECORD_ID to MOST_SEVERE_INJURY_REPORTED, NOT EVIDENT
dtypes: float64(2), int32(1), int64(122), object(2)
memory usage: 430.3+ MB


In [4]:
crashes_cleaned['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

FAILING TO YIELD RIGHT-OF-WAY                                                       87034
FOLLOWING TOO CLOSELY                                                               77351
IMPROPER OVERTAKING/PASSING                                                         39012
FAILING TO REDUCE SPEED TO AVOID CRASH                                              33568
IMPROPER BACKING                                                                    31286
IMPROPER LANE USAGE                                                                 28331
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                 26380
IMPROPER TURNING/NO SIGNAL                                                          26339
DISREGARDING TRAFFIC SIGNALS                                                        15440
WEATHER                                                                             11870
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER    10097
DISREGARDI

In [10]:
disregard_list = ['DISREGARDING TRAFFIC SIGNALS', 'DISREGARDING STOP SIGN', 'DISREGARDING ROAD MARKINGS', 
                  'DISREGARDING OTHER TRAFFIC SIGNS', 'DISREGARDING YIELD SIGN', 'DRIVING ON WRONG SIDE/WRONG WAY', 
                  'FAILING TO YIELD RIGHT-OF-WAY','FOLLOWING TOO CLOSELY', 'IMPROPER OVERTAKING/PASSING', 
                  'FAILING TO REDUCE SPEED TO AVOID CRASH', 'TURNING RIGHT ON RED', 'EXCEEDING SAFE SPEED FOR CONDITIONS', 
                  'EXCEEDING AUTHORIZED SPEED LIMIT', 'IMPROPER LANE USAGE', 'PHYSICAL CONDITION OF DRIVER', 
                  'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE','IMPROPER BACKING', 'IMPROPER TURNING/NO SIGNAL'] 

distraction_list = ['TEXTING', 'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)', 
                    'DISTRACTION - FROM INSIDE VEHICLE','CELL PHONE USE OTHER THAN TEXTING']

drinking_list = ['OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER', 
                 'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)', 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)']

road_list = ['DISTRACTION - FROM OUTSIDE VEHICLE', 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS', 'ROAD CONSTRUCTION/MAINTENANCE', 'EQUIPMENT - VEHICLE CONDITION', 
             'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)', 'WEATHER']

other_list= ['PASSING STOPPED SCHOOL BUS', 'OBSTRUCTED CROSSWALKS', 'BICYCLE ADVANCING LEGALLY ON RED LIGHT', 
             'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT', 'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST', 'ANIMAL', 'TURNING RIGHT ON RED', 
             'RELATED TO BUS STOP'] 

binning_list = [disregard_list, distraction_list, drinking_list, road_list, other_list]
value_list = ['DRIVER ERROR', 'DISTRACTION INSIDE VEHICLE', 'DRINKING/DRUGS', 'OUTSIDE FACTORS', 'OTHER']


for group, value in zip(binning_list, value_list):
    crashes_cleaned['PRIM_CONTRIBUTORY_CAUSE'] = crashes_cleaned['PRIM_CONTRIBUTORY_CAUSE'].replace(to_replace = group, value = value)

crashes_cleaned['PRIM_CONTRIBUTORY_CAUSE'].value_counts(normalize = True)


PRIM_CONTRIBUTORY_CAUSE
DRIVER ERROR                  0.880459
OUTSIDE FACTORS               0.063773
DRINKING/DRUGS                0.033166
DISTRACTION INSIDE VEHICLE    0.016300
OTHER                         0.006302
Name: proportion, dtype: float64

In [6]:

#crashes_cleaned.plot('PRIM_CONTRIBUTORY_CAUSE', kind= 'barh')

In [7]:
import numpy as np

corr_matrix = crashes_cleaned.select_dtypes(include=np.number).corr()

# Set a correlation threshold (e.g., 0.7 for highly correlated)
threshold = 0.6

# Get a list of highly correlated columns
highly_correlated = []

# Iterate through the correlation matrix and find highly correlated columns
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            colname = corr_matrix.columns[i]
            highly_correlated.append((corr_matrix.columns[j], colname))

print("Highly correlated columns:")
for col1, col2 in highly_correlated:
    print(f"{col1} and {col2}")




Highly correlated columns:
LATITUDE and LONGITUDE
WEATHER_CONDITION_CLEAR and WEATHER_CONDITION_RAIN
LIGHTING_CONDITION_DARKNESS, LIGHTED ROAD and LIGHTING_CONDITION_DAYLIGHT
ALIGNMENT_STRAIGHT AND LEVEL and ALIGNMENT_STRAIGHT ON GRADE
WEATHER_CONDITION_SNOW and ROADWAY_SURFACE_COND_SNOW OR SLUSH
WEATHER_CONDITION_CLEAR and ROADWAY_SURFACE_COND_WET
WEATHER_CONDITION_RAIN and ROADWAY_SURFACE_COND_WET
ROAD_DEFECT_NO DEFECTS and ROAD_DEFECT_UNKNOWN
CRASH_TYPE and MOST_SEVERE_INJURY_NO INDICATION OF INJURY
MOST_SEVERE_INJURY_NO INDICATION OF INJURY and MOST_SEVERE_INJURY_NONINCAPACITATING INJURY


In [8]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Separate the target variable 'Y' from predictor variables
X = crashes_cleaned.select_dtypes(include=np.number)

# Add a constant term (intercept) to the predictor variables
X = sm.add_constant(X)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif['Variable'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Print the VIF values
print(vif)


                                        Variable            VIF
0                                          const  166602.400596
1                             POSTED_SPEED_LIMIT       1.431306
2                                     CRASH_TYPE       2.150143
3                         INTERSECTION_RELATED_I       1.390872
4                             NOT_RIGHT_OF_WAY_I       1.422050
..                                           ...            ...
121                     ROAD_DEFECT_WORN SURFACE       6.070958
122     MOST_SEVERE_INJURY_INCAPACITATING INJURY      18.623686
123   MOST_SEVERE_INJURY_NO INDICATION OF INJURY     123.809263
124  MOST_SEVERE_INJURY_NONINCAPACITATING INJURY      76.624255
125     MOST_SEVERE_INJURY_REPORTED, NOT EVIDENT      43.091202

[126 rows x 2 columns]


In [9]:
crashes_cleaned.to_csv('../data/crashes_cleaned.csv', index=True)